In [ ]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import glob
import shutil
import json
import keras
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
from tensorflow.keras import layers
from tensorflow.keras import applications
from tensorflow.keras.layers import Conv2D,Dense, Dropout, BatchNormalization, GlobalAveragePooling2D,Input
from tqdm import tqdm
from tensorflow.keras.mixed_precision import experimental as mixed_precision
# Defining the working directories

work_dir= '../input/cassava-leaf-disease-classification/' 

work_dir_merged = '../input/cassava-leaf-disease-merged/'
train_path_merged = '/kaggle/input/cassava-leaf-disease-merged/train'

train_folder = "/kaggle/input/cassava-leaf-disease-merged/train/"

n_CLASS = 5
IMG_SIZE=320
BATCH_SIZE=32


data = pd.read_csv(work_dir_merged + 'merged.csv')
#data['label']=data['label'].astype(str)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

f = open(work_dir + 'label_num_to_disease_map.json')
real_labels = json.load(f)
real_labels = {int(k):v for k,v in real_labels.items()}

# Defining the working dataset
data['class_name'] = data.label.map(real_labels)
data['label']=data['label'].astype(str)
data['class_name']=data['class_name'].astype(str)

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy) #shortens training time by 2x

print(Counter(data['label'])) # Checking the frequencies of the labels

drop_indexs = pd.read_csv("../input/noisylabels/noise_labels.csv")
list_index_drops=product = drop_indexs['noise_labels'].values.tolist()

data = data.drop(list_index_drops, axis="index")
data.reset_index(drop=True, inplace=True)

print(Counter(data['label']))

# Spliting the data
from sklearn.model_selection import train_test_split

#train,val = train_test_split(data, test_size = 0.1, random_state = 42, stratify = data['class_name'])

# Importing the data using ImageDataGenerator

from keras.preprocessing.image import ImageDataGenerator

# Data agumentation and pre-processing using tensorflow
train_gen = ImageDataGenerator(
                                rotation_range=270,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                brightness_range=[0.1,0.9],
                                shear_range=25,
                                zoom_range=0.3,
                                channel_shift_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1/255,
                                validation_split=0.2
                               )
                                    
    
valid_gen = ImageDataGenerator(rescale=1/255,
                               validation_split = 0.2
                              )

train_generator = train_gen.flow_from_dataframe(
                            dataframe=data,
                            directory = train_path_merged,
                            x_col = "image_id",
                            y_col = "class_name",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "sparse",
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            subset = "training",

)

valid_generator = valid_gen.flow_from_dataframe(
                            dataframe=data,
                            directory = train_path_merged,
                            x_col = "image_id",
                            y_col = "class_name",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "sparse",
                            batch_size = BATCH_SIZE,
                            shuffle = False,
                            subset = "validation"
)

"""
base = applications.Xception(include_top=False, weights="imagenet",input_shape=[IMG_SIZE,IMG_SIZE,3])
for layer in base.layers[:90]:
    layer.trainable = False
for layer in base.layers[90:]:
    layer.trainable = True

base.summary()
model = tf.keras.Sequential()
model.add(base)
model.add(BatchNormalization(axis=-1))
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dense(5, activation='softmax'))

callback0 = tf.keras.callbacks.ModelCheckpoint("./CasavaLeafDiseaseModel_xception_freeze.h5", 
                                               monitor='val_loss',save_best_only=True)

model.compile(loss="sparse_categorical_crossentropy", optimizer="Adam", metrics=['acc'])
model.fit(train_generator, epochs=30, batch_size=BATCH_SIZE,validation_data=valid_generator,callbacks=[callback0])
"""
model=tf.keras.models.load_model('../input/xception-rmsprop-8epochs-cassave/CasavaLeafDiseaseModel_xception_freeze.h5')
for layer in model.layers:
    layer.trainable = True

callback1 = tf.keras.callbacks.ModelCheckpoint("./CasavaLeafDiseaseModel_xception_unfreeze.h5", 
                                               monitor='val_loss',save_best_only=True)

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(1e-5), metrics=['acc'])
model.fit(train_generator, epochs=20, batch_size=BATCH_SIZE,validation_data=valid_generator,callbacks=[callback1])
    
model.save("Xception_ImageNet_Adam_10epochs_nontrainable")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session